## Looking at the azimuth wrapping as a cause of oscillation fails

Craig Lage - 10-Apr-23

In [ ]:
import sys, time, os, asyncio
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler

In [ ]:
# Get EFD client and butler
client = EfdClient('summit_efd')
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib"])

In [ ]:
# Times to start looking at encoder values
start = Time("2023-04-04 14:12:00Z", scale='utc')
end = Time("2023-04-04 15:27:00Z", scale='utc')
plotStart = (start + TimeDelta(0.0, format='sec')).to_datetime()
plotEnd = (end + TimeDelta(0.0, format='sec')).to_datetime()

In [ ]:
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", "azimuthCalculatedAngle", start, end)
azs = az['azimuthCalculatedAngle'].values
times = az['times'].values
t0 = times[0]
times -= t0
tmax = times[-1]

az_torque_1 =  await client.select_packed_time_series("lsst.sal.ATMCS.measuredTorque", "azimuthMotor1Torque", start, end)
torques = az_torque_1['azimuthMotor1Torque'].values

In [ ]:
t0

In [ ]:
plt.clf()
fig, [ax1,ax2] = plt.subplots(2,1,figsize = (16,8))
plt.subplots_adjust(hspace=0.5)
ax1.set_title("Impact of unwrapping on mount motion", fontsize = 18)
ax1.plot(times, azs, color='green', label="Azimuth")
ax1.set_ylabel("Degrees")
ax1.set_xlabel("Time (seconds)")
ax3=ax1.twinx()

ax2.set_title("Azimuth Motor 1 Torque", fontsize=18)
ax2.plot(times, torques)
ax2.plot([times[0], times[-1]], [0.0,0.0], ls='--', color='red')
ax2.set_ylabel("Torque(amps)")
ax2.set_xlabel("Time (seconds)")


mount_times = []
mount_motions = []
for seqNum in df.index.values.tolist():
    if seqNum >= 14 and seqNum <= 86:
        time = Time(f"2023-04-04T{df.loc[seqNum]['TAI']}", format='isot', scale='tai')
        mount_times.append(time.unix_tai - t0)
        mount_motions.append(df.loc[seqNum]['Mount motion image degradation'])

ax3.scatter(mount_times, mount_motions, marker='x', label="Mount motion")
ax3.set_ylim(0, 2.5)
ax3.set_ylabel("Mount motion image degradation (arcseconds)")
        
ax1.legend(loc="upper left")
ax3.legend()
plt.savefig(f"/home/craiglagegit/DATA/stiction_04apr23/Unwrapping_10Apr23.png")


In [ ]:
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib"])

In [ ]:
torques = []
mount_motions = []
for seqNum in df.index.values.tolist():
    if seqNum >= 14 and seqNum <= 86:
        
        expId = 2023040400000 + seqNum
        mData = butler.get('raw.metadata', detector=0, exposure=expId)
        tstart = Time(mData['DATE-BEG'], scale='tai')
        tend = tstart + TimeDelta(30.0, format='sec')
        
        az_torque =  await client.select_packed_time_series("lsst.sal.ATMCS.measuredTorque", "azimuthMotor1Torque", tstart.utc, tend.utc)
        ave_torque = np.mean(abs(az_torque['azimuthMotor1Torque'].values))
        #print(expId, tstart, tend, max_torque)
        torques.append(ave_torque)
        mount_motions.append(df.loc[seqNum]['Mount motion image degradation'])

        
        
        

In [ ]:
plt.scatter(torques, mount_motions)
plt.title("Mount Errors vs tracking torque", fontsize=18)
plt.xlabel("Mean(abs(azimuth torque while tracking)) (amps)")
plt.ylabel("Mount motion image degradation(arcseconds)")
plt.savefig(f"/home/craiglagegit/DATA/stiction_04apr23/Errors_vs_Torque_10Apr23.png")

In [ ]:
velocities = []
mount_motions = []
for seqNum in df.index.values.tolist():
    if seqNum >= 14 and seqNum <= 86:
        
        expId = 2023040400000 + seqNum
        mData = butler.get('raw.metadata', detector=0, exposure=expId)
        tstart = Time(mData['DATE-BEG'], scale='tai')
        tend = tstart + TimeDelta(30.0, format='sec')
        
        az_velocity =  await client.select_packed_time_series("lsst.sal.ATMCS.measuredMotorVelocity", "azimuthMotor1Velocity", tstart.utc, tend.utc)
        ave_velocity = np.mean(abs(az_velocity['azimuthMotor1Velocity'].values))
        #print(expId, tstart, tend, max_torque)
        velocities.append(ave_velocity)
        mount_motions.append(df.loc[seqNum]['Mount motion image degradation'])

        
        
        

In [ ]:
plt.scatter(velocities, mount_motions)
plt.title("Mount Errors vs azimuth velocity", fontsize=18)
plt.xlabel("Mean(abs(azimuth velocity while tracking)) (deg/sec)")
plt.ylabel("Mount motion image degradation(arcseconds)")
plt.savefig(f"/home/craiglagegit/DATA/stiction_04apr23/Errors_vs_Velocity_10Apr23.png")

In [ ]:
# Times to start looking at encoder values
start = Time("2023-04-11 16:51:00Z", scale='utc')
end = Time("2023-04-11 18:17:00Z", scale='utc')
plotStart = (start + TimeDelta(0.0, format='sec')).to_datetime()
plotEnd = (end + TimeDelta(0.0, format='sec')).to_datetime()

In [ ]:
# Get Rubin TV JSON
filename = f'/scratch/cslage/starTracker/auxtel_2023-04-11.json'
df = pd.read_json(filename)
df = df.transpose()


In [ ]:
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", "azimuthCalculatedAngle", start, end)
azs = az['azimuthCalculatedAngle'].values
times = az['times'].values
t0 = times[0]
times -= t0
tmax = times[-1]

az_torque_1 =  await client.select_packed_time_series("lsst.sal.ATMCS.measuredTorque", "azimuthMotor1Torque", start, end)
torques = az_torque_1['azimuthMotor1Torque'].values

In [ ]:
plt.clf()
fig, [ax1,ax2] = plt.subplots(2,1,figsize = (16,8))
plt.subplots_adjust(hspace=0.5)
ax1.set_title("Impact of unwrapping on mount motion", fontsize = 18)
ax1.plot(times, azs, color='green', label="Azimuth")
ax1.set_ylabel("Degrees")
ax1.set_xlabel("Time (seconds)")
ax3=ax1.twinx()

ax2.set_title("Azimuth Motor 1 Torque", fontsize=18)
ax2.plot(times, torques)
ax2.plot([times[0], times[-1]], [0.0,0.0], ls='--', color='red')
ax2.set_ylabel("Torque(amps)")
ax2.set_xlabel("Time (seconds)")


mount_times = []
mount_motions = []
for seqNum in df.index.values.tolist():
    if seqNum >= 6 and seqNum <= 97:
        time = Time(f"2023-04-11T{df.loc[seqNum]['TAI']}", format='isot', scale='tai')
        mount_times.append(time.unix_tai - t0)
        mount_motions.append(df.loc[seqNum]['Mount motion image degradation'])

ax3.scatter(mount_times, mount_motions, marker='x', label="Mount motion")
ax3.set_ylim(0, 2.5)
ax3.set_ylabel("Mount motion image degradation (arcseconds)")
        
ax1.legend(loc="upper left")
ax3.legend()
plt.savefig(f"/home/craiglagegit/DATA/stiction_11apr23/Unwrapping_11Apr23.png")


In [ ]:
torques = []
mount_motions = []
for seqNum in df.index.values.tolist():
    if seqNum >= 6 and seqNum <= 97:
        
        expId = 2023041100000 + seqNum
        mData = butler.get('raw.metadata', detector=0, exposure=expId)
        tstart = Time(mData['DATE-BEG'], scale='tai')
        tend = tstart + TimeDelta(30.0, format='sec')
        
        az_torque =  await client.select_packed_time_series("lsst.sal.ATMCS.measuredTorque", "azimuthMotor1Torque", tstart.utc, tend.utc)
        ave_torque = np.mean(abs(az_torque['azimuthMotor1Torque'].values))
        #print(expId, tstart, tend, max_torque)
        torques.append(ave_torque)
        mount_motions.append(df.loc[seqNum]['Mount motion image degradation'])

        
        
        

In [ ]:
plt.scatter(torques, mount_motions)
plt.title("Mount Errors vs tracking torque", fontsize=18)
plt.xlabel("Mean(abs(azimuth torque while tracking)) (amps)")
plt.ylabel("Mount motion image degradation(arcseconds)")
plt.savefig(f"/home/craiglagegit/DATA/stiction_11apr23/Errors_vs_Torque_11Apr23.png")

In [ ]:
velocities = []
mount_motions = []
for seqNum in df.index.values.tolist():
    if seqNum >= 14 and seqNum <= 86:
        
        expId = 2023040400000 + seqNum
        mData = butler.get('raw.metadata', detector=0, exposure=expId)
        tstart = Time(mData['DATE-BEG'], scale='tai')
        tend = tstart + TimeDelta(30.0, format='sec')
        
        az_velocity =  await client.select_packed_time_series("lsst.sal.ATMCS.measuredMotorVelocity", "azimuthMotor1Velocity", tstart.utc, tend.utc)
        ave_velocity = np.mean(abs(az_velocity['azimuthMotor1Velocity'].values))
        #print(expId, tstart, tend, max_torque)
        velocities.append(ave_velocity)
        mount_motions.append(df.loc[seqNum]['Mount motion image degradation'])

        
        
        

In [ ]:
plt.scatter(velocities, mount_motions)
plt.title("Mount Errors vs azimuth velocity", fontsize=18)
plt.xlabel("Mean(abs(azimuth velocity while tracking)) (deg/sec)")
plt.ylabel("Mount motion image degradation(arcseconds)")
plt.savefig(f"/home/craiglagegit/DATA/stiction_04apr23/Errors_vs_Velocity_10Apr23.png")